In [1]:
import cv2
import numpy as np
from scipy import ndimage
import requests

In [2]:
response = requests.get("https://raw.githubusercontent.com/PacktPublishing/Learning-OpenCV-4-Computer-Vision-with-Python-Third-Edition/master/images/statue_small.jpg",
                        stream=True)
if response.status_code == 200:
    with open("../images/statue.jpg",'wb') as f:
        for chunk in response.iter_content(chunk_size=1024):
            f.write(chunk)

# High Pass Filter and Low Pass Filter

In [3]:
kernel_3x3  = np.full(shape=(3,3),fill_value=-1)
kernel_3x3[1,1] = 8
kernel_3x3

array([[-1, -1, -1],
       [-1,  8, -1],
       [-1, -1, -1]])

In [4]:
kernel_5x5 = np.full(shape=(5,5),fill_value=-1)
kernel_5x5[1:4,1:4] = np.array([[1,2,1],
                                [2,4,2],
                                [1,2,1]])
kernel_5x5

array([[-1, -1, -1, -1, -1],
       [-1,  1,  2,  1, -1],
       [-1,  2,  4,  2, -1],
       [-1,  1,  2,  1, -1],
       [-1, -1, -1, -1, -1]])

In [50]:
def view_image(window_name,img,destroy=False):
    cv2.namedWindow(window_name,cv2.WINDOW_NORMAL)
    cv2.imshow(window_name,img)
    if destroy:
        cv2.waitKey()
        cv2.destroyAllWindows()

In [6]:
img = cv2.imread("../images/statue.jpg",cv2.IMREAD_GRAYSCALE)
k3 = ndimage.convolve(img,kernel_3x3)
k5 = ndimage.convolve(img,kernel_5x5)

blurred = cv2.GaussianBlur(img,(17,17),11)
g_hpf = img - blurred

view_image("3x3",k3)
view_image("5x5",k5)
view_image("blurred",blurred)
view_image("g_hpf",g_hpf)
cv2.waitKey()
cv2.destroyAllWindows()

# Edge Detection

1. OpenCV provides many edge-finding filters including 
   1. Laplacian
   2. Sobel
   3. Scharr
2. These filter turn non0edge region into black and turn the edge region into white.
3. However these filter are prone to capture the noise as edge.
4. To overcome the drawback we will blur to image to denoise  the image.

In [28]:
def stroke_edge(src,dst,blur_ksize = 7, edge_ksize = 5 ):
    view_image("source",src)
    # apply blur image
    if blur_ksize >= 3:
        blurred_src = cv2.medianBlur(src,blur_ksize)
        view_image("blur",blurred_src)
        gray_src = cv2.cvtColor(blurred_src,code=cv2.COLOR_BGR2GRAY)
    else:
        gray_src = cv2.cvtColor(src,code=cv2.COLOR_BGR2GRAY)
    view_image("gray",gray_src)
    # apply edge detector
    cv2.Laplacian(gray_src,cv2.CV_8U,gray_src,edge_ksize)
    view_image("Laplacian",gray_src)
    normalized_inverse_alpha = (1.0/255) * (255 - gray_src)
    channels = cv2.split(src)
    for channel  in channels:
        channel[:] = channel * normalized_inverse_alpha
    cv2.merge(channels,dst)   
    
    view_image("final",dst)
    cv2.waitKey()
    cv2.destroyAllWindows()

In [29]:
img = cv2.imread("../images/statue.jpg")

In [30]:
stroke_edge(img,img)

In [36]:
view_image("sfd",img)
cv2.waitKey()
cv2.destroyAllWindows()

# Custom Kernel

## Sharpen kernel

When the weight kernel sum to 1. then we have the sharpen kernel.

In [59]:
# central element in the kernel matrix is pixel of interest
sharp_kernel = np.full(shape=(3,3),fill_value=-1)
sharp_kernel[1,1] = 9
sharp_kernel
#? if the pixel of interest is already different from its neighbor ,
#?  then this difference is intensified

array([[-1, -1, -1],
       [-1,  9, -1],
       [-1, -1, -1]])

In [60]:
np.sum(sharp_kernel)

1

In [61]:
modified_sharpen_kernel = np.full(shape=(3,3),fill_value=-2)
modified_sharpen_kernel[1,1] = 17
modified_sharpen_kernel

array([[-2, -2, -2],
       [-2, 17, -2],
       [-2, -2, -2]])

In [70]:
img = cv2.imread("../images/statue.jpg")

In [71]:
view_image("Sharpen",cv2.filter2D(img,-1,sharp_kernel))
view_image("modified_sharpen_kernel",cv2.filter2D(img,-1,modified_sharpen_kernel))
cv2.waitKey()
cv2.destroyAllWindows()

## Edge Filter

In [72]:
edge_kernel = np.full(shape=(3,3),fill_value=-1)
edge_kernel[1,1] = 8
edge_kernel

array([[-1, -1, -1],
       [-1,  8, -1],
       [-1, -1, -1]])

In [74]:
view_image("original",img)
view_image("Sharpen",cv2.filter2D(img,-1,edge_kernel))
cv2.waitKey()
cv2.destroyAllWindows()

## Blur Filter

In [45]:
blur_kernel = np.full(shape=(5,5),fill_value=0.04)
blur_kernel

array([[0.04, 0.04, 0.04, 0.04, 0.04],
       [0.04, 0.04, 0.04, 0.04, 0.04],
       [0.04, 0.04, 0.04, 0.04, 0.04],
       [0.04, 0.04, 0.04, 0.04, 0.04],
       [0.04, 0.04, 0.04, 0.04, 0.04]])

In [51]:
view_image("Blur",cv2.filter2D(img,-1,blur_kernel),destroy=True)

## Emboss Filter

In [52]:
emboss_kernel = np.array([[-2,-1,0],
                          [-1,1,1],
                          [0,1,2]])
emboss_kernel

array([[-2, -1,  0],
       [-1,  1,  1],
       [ 0,  1,  2]])

In [53]:
view_image("emboss",cv2.filter2D(img,-1,emboss_kernel),destroy=True)

## Canny Edge detection

In [85]:
view_image("original",img)
view_image("Canny",cv2.Canny(img,200,300))
cv2.waitKey()
cv2.destroyAllWindows()